In [48]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)


# Initialize Data Frame

In [5]:
testRaw = pd.read_csv('c:/Users/jmeis/NYC_DSA/HousingPrices/Data/test.csv')
testRaw.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


##  Find NAs

In [8]:
testRaw.isna().sum(axis=0)

Id                  0
MSSubClass          0
MSZoning            4
LotFrontage       227
LotArea             0
Street              0
Alley            1352
LotShape            0
LandContour         0
Utilities           2
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         1
Exterior2nd         1
MasVnrType         16
MasVnrArea         15
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           44
BsmtCond           45
BsmtExposure       44
BsmtFinType1       42
BsmtFinSF1          1
BsmtFinType2       42
BsmtFinSF2          1
BsmtUnfSF           1
TotalBsmtSF         1
Heating             0
HeatingQC           0
CentralAir          0
Electrical          0
1stFlrSF            0
2ndFlrSF            0
LowQualFin

## Action Plan for NA values

**Cleaning NAs is going to be a process**  
+ All variables with quality in the description use NA to denote there is no object to denote the quality of  
    - suggest replacing with text 'none' and using that as the default case in dummification   
    - build a masking variable
+ MSZoning has 4 NA values, no easily identifiable characteristics to interpolate, strongly consider dropping. Could be a very important variable.
+ LotFrontage Variable has 227 NA values, but no values equal to zero, assuming NA means 0
+ Utilities has 2 NA values, no identifying charactersitics for this variable strongly consider dropping  
+ Exterior1st has 1 NA value, convert to 'Other' 
+ Exterior2nd has 1 NA value, convert to 'Other' 
+ MasVnrArea has 16 NA values, convert to None  
+ MasVnrQual has 15 NA values corresponding to the NAs in previous category, change to 0 (an acceptable value in this dataset)
+ BsmtFinSF1 has 1 NA value, change to 0 (see anaomlies section for explanation)  
+ BsmtFinSF2 has 1 NA value, change to 0 (see anaomlies section for explanation)  
+ BsmtUnfSF has 1 NA value, change to 0 (see anaomlies section for explanation)  
+ BsmtFullBath has 2 NAs, but records say there is no basement, change to zero  
+ BsmtHalfBath has 2 NAs, but records say there is no basement, change to zero 
+ KitchenAbvGr has 1 NA, but has a kitchen listed in the record, no way to interpolate, consider dropping
+ Functional has 2 NA values, no way to interpolate consider dropping
+ GarageCars, and GarageArea NA values will be dropped (see below anomoly description)
+ SaleType, has 1 NA value, change to 'Oth'

**Variables with NA values who's rows we should drop**  
+ MSZoning  
+ Utilities
+ KitchenQual
+ Functional




### Mask Variable for replacing NA values in columns where NA has meaning

In [73]:
naMask = ['Alley', 'Exterior1st','Exterior2nd', 'BsmtQual', 'BsmtCond','BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish'\
         ,'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature' ]

### Additional Data Anomolies

**Mismatch in Basement Existence Data**

In [37]:
testRaw[testRaw['BsmtQual'].isna()].loc[:,['BsmtQual', 'BsmtCond','BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1','BsmtFinType2', 'BsmtFinSF2','BsmtUnfSF', 'TotalBsmtSF' ]]\
.loc[testRaw['BsmtUnfSF']!=0]

,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF
660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
757,NaN,Fa,No,Unf,0.0,Unf,0.0,173.0,173.0
758,NaN,TA,No,Unf,0.0,Unf,0.0,356.0,356.0


These 3 records have basement quality listed as 'No Basment', the last two records however have values listed for basement features, can not interpolate basement quality from data recomend dropping records 757 and 758.
For record 660 Set basement square footage columns to 0, replacing NA values to NONE in previous step will take care of the rest.

**Missing Column, Kitchen**

In the data dictionary file, a column named 'Kitchen' is listed, however the datafile has a column named 'KitchenAbvGr' instead

**Garage Existence Data Mismatch**

In [53]:
testRaw[testRaw['GarageYrBlt'].isna()].loc[~testRaw['GarageType'].isna(), ['GarageType','GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'GarageYrBlt']]

,GarageType,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,GarageYrBlt
666,Detchd,NaN,1.0,360.0,NaN,NaN,NaN
1116,Detchd,NaN,NaN,NaN,NaN,NaN,NaN


2 records with garage existence listed but limited values listed for the rest with no legal option for NaN, consider dropping records 666, 1116

# Clean Data

## Drop Problematic records by row index
**Previously documented 4 records with uniterpolatable illegal NA values**


In [92]:
print(testRaw.shape)
testClean = testRaw.drop([757, 758, 666, 1116])
testClean.shape

(1459, 80)


(1455, 80)

## Drop Rows with Ilegal NA values in Suspected High Importance Columns

In [93]:
dropImportant = ['MSZoning','Utilities','KitchenQual','Functional']

In [94]:
testClean = testClean.loc[~testClean['MSZoning'].isna() & ~testClean['Utilities'].isna() & ~testClean['KitchenQual'].isna() & ~testClean['Functional'].isna()]
testClean.shape

(1448, 80)

## Change NA value of certain columns where NA has meaning to None

In [95]:
for i in range(len(naMask)):
    testClean[naMask[i]].fillna('None', inplace = True)

## Change Specific NA values depending on Context  

**LotFrontage Variable has 227 NA values, but no values equal to zero, assuming NA means 0**

In [96]:
testClean.LotFrontage.fillna(0, inplace = True)

**Exterior1st has 1 NA value, convert to 'Other'**

In [97]:
testClean.Exterior1st.fillna('Other', inplace = True)

**Exterior2nd has 1 NA value, convert to 'Other'**

In [98]:
testClean.Exterior2nd.fillna('Other', inplace = True)

**MasVnrType has 16 NA values, convert to None**

In [99]:
testClean.MasVnrType.fillna('None', inplace = True)

**MasVnrArea has 15 NA values corresponding to the NAs in previous category, change to 0**

In [100]:
testClean.MasVnrArea.fillna(0, inplace = True)

**TotalBsmtSF has 1 NA value, change to 0**

In [110]:
testClean.TotalBsmtSF.fillna(0, inplace = True)

**BsmtFinSF1 has 1 NA value, change to 0**

In [101]:
testClean.BsmtFinSF1.fillna(0, inplace = True)

**BsmtFinSF2 has 1 NA value, change to 0**

In [102]:
testClean.BsmtFinSF2.fillna(0, inplace = True)

**BsmtUnfSF has 1 NA value, change to 0**

In [103]:
testClean.BsmtUnfSF.fillna(0, inplace = True)

**BsmtFullBath has 2 NAs, but records say there is no basement, change to zero**

In [104]:
testClean.BsmtFullBath.fillna(0, inplace = True)

**BsmtHalfBath has 2 NAs, but records say there is no basement, change to zero**

In [105]:
testClean.BsmtHalfBath.fillna(0, inplace = True)

**SaleType, has 1 NA value, change to 'Oth'**

In [106]:
testClean.SaleType.fillna('Oth', inplace = True)

## Check to see if we elimanted all NA values

In [111]:
testClean.isna().sum(axis=0)

Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
Street           0
Alley            0
LotShape         0
LandContour      0
Utilities        0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
MasVnrArea       0
ExterQual        0
ExterCond        0
Foundation       0
BsmtQual         0
BsmtCond         0
BsmtExposure     0
BsmtFinType1     0
BsmtFinSF1       0
BsmtFinType2     0
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
Heating          0
HeatingQC        0
CentralAir       0
Electrical       0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
BsmtFullBath     0
BsmtHalfBath     0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr

## A Thing of Beauty Emerges, no NA values, save to new csv

In [113]:
testClean.to_csv('c:/Users/jmeis/NYC_DSA/HousingPrices/Data/testZeroNAs.csv')

# Simple Feature Selection